# Imports


In [ ]:
import os

from langchain_ollama import ChatOllama

os.environ["NO_PROXY"] = "localhost,127.0.0.1"
os.environ["USER_AGENT"] = "myagent"

# LLM Initialisierung mit Beispiel


In [ ]:
llm = ChatOllama(
    model="gpt-oss:20b-cloud",
    base_url="http://localhost:11434",
    temperature=0.7,
)

messages = [
    ("system", "Du bist ein hilfreicher Assistent."),
    ("human", "Erkläre mir die Vorteile eines lokalen LLM."),
]

response = llm.invoke(messages)
print(response.content)

# Embedding Modell Initialisierung


# Vektordatenbank Initialisierung

### Eine Collection für eine Website und eine für ein PDF-Dokument erstellen. Anschließend Collections anzeigen lassen.


### Optional: Vorherige Sammlung löschen


In [ ]:
client.delete_collection(collection_name=name_der_sammlung)

# Website laden und Text extrahieren (Website)


### Text in Chunks aufteilen (Website)


### Chunks in Vektordatenbank speichern (Website)


# PDF laden und Text extrahieren (PDF)


### Text in Chunks aufteilen (PDF)


### Chunks in Vektordatenbank speichern (PDF)


# LLM mit Retrieval Augmented Generation verwenden


### Website


### PDF
